In [1]:
import pandas as pd
import networkx as nx
import community as community_louvain
from collections import Counter

# === Cargar nodos y aristas ===
nodes_df = pd.read_csv("../data/networks/lab_nodes.csv")
edges_df = pd.read_csv("../data/networks/lab_edges.csv")

# === Construir grafo no dirigido ===
G = nx.Graph()
for _, row in nodes_df.iterrows():
    G.add_node(row["Id"], label=row["Label"], lab=row["Lab"])
for _, row in edges_df.iterrows():
    G.add_edge(row["Source"], row["Target"], weight=row["Weight"])

print(G.number_of_nodes())
print(G.number_of_edges())

# === Probar distintos valores de resolution ===
resolutions = [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0]

print("=== Louvain con distintos resolution ===\n")

for res in resolutions:
    partition = community_louvain.best_partition(G, weight='weight', resolution=res)
    modularity = community_louvain.modularity(partition, G, weight='weight')
    num_communities = len(set(partition.values()))
    sizes = Counter(partition.values())

    print(f"🧪 Resolution = {res}")
    print(f"  - Número de comunidades: {num_communities}")
    print(f"  - Modularidad: {modularity:.4f}")
    print(f"  - Tamaños: {dict(sizes)}\n")

29
62
=== Louvain con distintos resolution ===

🧪 Resolution = 0.25
  - Número de comunidades: 10
  - Modularidad: 0.7718
  - Tamaños: {4: 4, 6: 3, 2: 3, 3: 3, 8: 6, 7: 3, 0: 1, 1: 2, 5: 2, 9: 2}

🧪 Resolution = 0.5
  - Número de comunidades: 10
  - Modularidad: 0.7718
  - Tamaños: {4: 4, 6: 3, 2: 3, 9: 3, 8: 6, 0: 3, 1: 1, 3: 2, 5: 2, 7: 2}

🧪 Resolution = 0.75
  - Número de comunidades: 8
  - Modularidad: 0.7825
  - Tamaños: {0: 4, 1: 5, 2: 6, 4: 6, 5: 3, 6: 1, 3: 2, 7: 2}

🧪 Resolution = 1.0
  - Número de comunidades: 8
  - Modularidad: 0.7825
  - Tamaños: {0: 4, 7: 5, 3: 6, 4: 6, 5: 3, 6: 1, 1: 2, 2: 2}

🧪 Resolution = 1.5
  - Número de comunidades: 9
  - Modularidad: 0.7763
  - Tamaños: {0: 4, 7: 5, 2: 3, 3: 3, 4: 6, 5: 3, 6: 1, 8: 2, 1: 2}

🧪 Resolution = 2.0
  - Número de comunidades: 10
  - Modularidad: 0.7718
  - Tamaños: {3: 4, 5: 3, 2: 3, 9: 3, 8: 6, 6: 3, 0: 1, 1: 2, 4: 2, 7: 2}

🧪 Resolution = 3.0
  - Número de comunidades: 10
  - Modularidad: 0.7718
  - Tamaños: {4: 4, 1:

In [2]:
# === Buscar la mejor partición (mayor modularidad)
best_partition = None
best_modularity = -1
best_res = None

for res in resolutions:
    partition = community_louvain.best_partition(G, weight='weight', resolution=res)
    modularity = community_louvain.modularity(partition, G, weight='weight')
    
    if modularity > best_modularity:
        best_partition = partition
        best_modularity = modularity
        best_res = res

grupo_forzado = [
    "Sancho López, Jaime",
    "Zubiaur, Mercedes",
    "Matesanz del Barrio, Fuencisla",
    "Alcina, Antonio"
]

# === Forzar comunidad manualmente para los nodos seleccionados
# Tomamos la comunidad más común entre ellos, o una nueva comunidad
comunidad_objetivo = min(
    (best_partition[n] for n in grupo_forzado if n in best_partition),
    default=max(best_partition.values()) + 1
)

for nodo in grupo_forzado:
    if nodo in best_partition:
        best_partition[nodo] = comunidad_objetivo

# Guardar en CSV
df_partition = pd.DataFrame.from_dict(best_partition, orient='index', columns=['lovaina_community'])
df_partition.index.name = 'author_name'
df_partition.reset_index(inplace=True)
df_partition.to_csv("LovainaLeiden/louvain_best_partition.csv", index=False)

print(f"\n✅ Mejor partición guardada (res={best_res}, modularidad={best_modularity:.4f}) en 'output/louvain_best_partition.csv'")


✅ Mejor partición guardada (res=0.75, modularidad=0.7825) en 'output/louvain_best_partition.csv'


In [3]:
from pyvis.network import Network
import networkx as nx
from IPython.display import display, IFrame


# Crear red
net = Network(height='700px', width='100%', notebook=True, directed=False)
net.barnes_hut()

net.set_options("""
{
  "nodes": {
    "font": {
      "size": 16,
      "face": "Tahoma"
    },
    "shape": "dot"
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "interaction": {
    "hover": true,
    "tooltipDelay": 200
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -12000,
      "springLength": 250,
      "springConstant": 0.02,
      "damping": 0.6
    },
    "minVelocity": 0.75,
    "stabilization": {
      "iterations": 250
    }
  }
}
""")


# Añadir nodos
for node in G.nodes():
    community = best_partition.get(node, -1)
    net.add_node(
        node,
        label=node,  # Se mostrará junto al nodo
        title=f"{node} (Comunidad {community})",  # Se mostrará en tooltip
        group=community,
        size=20
    )


# Añadir aristas
for u, v, data in G.edges(data=True):
    net.add_edge(u, v, value=data.get('weight', 1))

# Exportar y mostrar inline
file_path = "LovainaLeiden/lovaina_comunidades.html"
net.show(file_path)

# Inyectar título al HTML generado
with open(file_path, "r", encoding="utf-8") as f:
    html = f.read()

# Añadir título justo debajo de <body>
titulo_html = "<h2 style='text-align:center; font-family:Tahoma;'>Red de Coautorías por Comunidad (7 comunidades)</h2>\n"
html = html.replace("<body>", f"<body>\n{titulo_html}")

with open(file_path, "w", encoding="utf-8") as f:
    f.write(html)

# Mostrar en el notebook directamente
display(IFrame(src=file_path, width="100%", height="700px"))

LovainaLeiden/lovaina_comunidades.html
